In [3]:
#импортирование нужных библиотек
import pandas as pd # для анализа данных
from datetime import datetime # для перевода строк, обозначающих даты, в формат date
import math # использование математических операций
import numpy as np # поддержка многомерных массивов
from sklearn.model_selection import train_test_split # для разделения на тестовую и обучающую выборку
from sklearn.linear_model import LogisticRegression # обучение логистической регрессии 
from sklearn.metrics import confusion_matrix # матрица итогов прогнозирования
from sklearn.metrics import classification_report # подсчёт метрик
from sklearn.preprocessing import LabelEncoder,OneHotEncoder # декодинг данных в числа

In [4]:
# Файл с информацией о клиентах
first_file = pd.read_csv('resto-asap2018\CASC_Constant.csv')  # считываем данные из файла
print(first_file.head()) # отображаем верхушку данных
print(first_file.info()) # отображаем информацию о данных

   CustomerId ActivationDate   Age     Sex  SubscribedEmail  SubscribedPush
0     2728183     2015-01-01  21.0  Female            False            True
1     2728198     2015-01-01  21.0  Female             True            True
2     2728306     2015-01-01  21.0  Female             True            True
3     2728178     2015-01-01  22.0    Male             True            True
4     2728322     2015-01-01  22.0    Male             True            True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   ActivationDate   10000 non-null  object 
 2   Age              9953 non-null   float64
 3   Sex              8874 non-null   object 
 4   SubscribedEmail  10000 non-null  bool   
 5   SubscribedPush   10000 non-null  bool   
dtypes: bool(2), float64(1), int64(1), object(2)
memory usage: 332.2+

In [5]:
# Файл с информацией о походах клиентов в рестораны
second_file = pd.read_csv('resto-asap2018\casc-resto.csv', sep=';', low_memory=False, decimal=',') # разделитель ;, 
# не переопределяем значения, символ распознавания десятичных частей ","
print(second_file.head()) # отображаем верхушку данных
print(second_file.info()) # отображаем информацию о данных

   CustomerID  Restaurant      RKDate                    RegionName  \
0     2898197         391  2017-07-16  Москва и Московская область    
1     2903215          43  2015-04-07  Москва и Московская область    
2     2748887          43  2015-05-22  Москва и Московская область    
3     2862077          46  2015-03-05  Москва и Московская область    
4     2862077          46  2015-03-11  Москва и Московская область    

   BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  
0  TGI FRIDAYS      NON ALCOHOL         2        2.0                   2.0  
1     IL Патио      NON ALCOHOL         1        5.0                   5.0  
2     IL Патио      NON ALCOHOL         1        5.0                   5.0  
3     IL Патио      NON ALCOHOL         1        5.0                   5.0  
4     IL Патио      NON ALCOHOL         1        5.0                   5.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882222 entries, 0 to 882221
Data columns (total 9 columns):
 

In [6]:
# Файл с информацией о походах клиентов в рестораны (сжатая версия, далее не используется)
third_file = pd.read_csv('resto-asap2018\casc-resto-small.csv')
#print(third_file.head())
#print(third_file.info())

In [7]:
# Определить, совершал ли клиент визит в период между 2017-07-01 и 2017-12-31
start_date = datetime(2017,7,1) # зафиксированная дата 2017-07-01
final_date = datetime(2017,12,31) # зафикисированная дата 2017-12-31
second_file['RKDate'] = pd.to_datetime(second_file['RKDate']) # переводим данные из колонки RKDate из формата str в datetime
grouped = second_file.groupby(by='CustomerID',sort=False) # группируем по покупателям без сортировки
second_file['BetweenTwoDates']=grouped['RKDate'].apply(lambda x: (x>=start_date)&(x<=final_date)) # записываем в новый столбец
# результаты группировки с помощью выражения (True или False)
second_file = second_file.reset_index() # индекс по умолчанию
print(second_file.head())

   index  CustomerID  Restaurant     RKDate                    RegionName  \
0      0     2898197         391 2017-07-16  Москва и Московская область    
1      1     2903215          43 2015-04-07  Москва и Московская область    
2      2     2748887          43 2015-05-22  Москва и Московская область    
3      3     2862077          46 2015-03-05  Москва и Московская область    
4      4     2862077          46 2015-03-11  Москва и Московская область    

   BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  \
0  TGI FRIDAYS      NON ALCOHOL         2        2.0                   2.0   
1     IL Патио      NON ALCOHOL         1        5.0                   5.0   
2     IL Патио      NON ALCOHOL         1        5.0                   5.0   
3     IL Патио      NON ALCOHOL         1        5.0                   5.0   
4     IL Патио      NON ALCOHOL         1        5.0                   5.0   

   BetweenTwoDates  
0             True  
1            False  
2    

In [8]:
# Расчёт Recency - количество дней до последнего визита перед зафиксированной датой
def recency(dataframe):
    get_dates = dataframe[dataframe.RKDate<start_date].RKDate # берём даты, более ранние чем зафиксированная
    if get_dates.size==0: # если таких дат нет
        return 0 # возвращаем 0
    else:
        max_dates = get_dates.max() # берём максимальную дату из выбранных
        return (start_date - max_dates).days # возвращаем количество дней между последним посещением и ЗД

In [9]:
# расчёт frequency - среднее количество посещений в месяц
def frequency(dataframe):
    first_visit = dataframe['RKDate'].min() # берём самый ранний визит
    if first_visit >= start_date: # если он был позже ЗД
        return 0 # возвращаем 0
    days_count = (start_date - first_visit).days # берём количество дней между ЗД и первым визитом
    month_count = math.ceil(days_count / 30) # считаем количество прошедших месяцев
    visit_count = dataframe.RKDate.unique().size # считаем кол-во визитов (в один день можно сходить только один раз)  
    return visit_count/month_count # среднее количество равно количество визитов/количество месяцев

In [10]:
# расчёт monetary_value - средний чек клиента
def monetary_value(dataframe):
    correct_dates = dataframe[dataframe.RKDate < start_date] # берём даты раньше чем ЗД
    unique_dates = correct_dates.RKDate.unique() # берём только уникальные даты (1 день -1 посещение)
    if unique_dates.size == 0: # если таких дат нет - 0
        return 0
    else:
        point_usage_sum = correct_dates.SummAfterPointsUsage.sum() # считаем сумму записей за все даты 
        return point_usage_sum / unique_dates.size # среднее количество равно весь чек/кол-во дней

In [11]:
# сумма скидок при всех посещениях - какова экономия за всё время посещения
def point_usage(dataframe):    
    get_dates = dataframe[dataframe.RKDate<start_date] # берём даты раньше чем ЗД 
    basic_sum = get_dates.SummBasic.sum() # сумма всех чеков без скидки
    point_usage_sum = get_dates.SummAfterPointsUsage.sum() # сумма всех чеков со скидкой
    return basic_sum - point_usage_sum # вычисляем разницу

In [12]:
# формирование таблицы
def all_analytics(dataframe):
    coloumns = dict({ # создание словаря
        "Recency": recency(dataframe),
        "Frequency": frequency(dataframe),
        "Monetary value": monetary_value(dataframe),
        "Point usage": point_usage(dataframe)
    })
    return pd.Series(coloumns) # возвращаем структуру Series

In [13]:
# формирование DataFrame с нужными для модели данными
customers_analytics = second_file.groupby(by = "CustomerID", sort=False).apply(all_analytics) # возвращаем собранные ранее
# колонки с группировкой по покупателям
print(customers_analytics.head())
print(customers_analytics.info())

            Recency  Frequency  Monetary value  Point usage
CustomerID                                                 
2898197        64.0   0.964286     2695.222222       4754.0
2903215       367.0   0.285714     1697.875000       1057.0
2748887        23.0   1.833333     1241.320000       7020.0
2862077       761.0   0.724138      565.000000       1123.0
2801997       458.0   0.517241      580.033333       1347.5
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2898197 to 2744732
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Recency         10000 non-null  float64
 1   Frequency       10000 non-null  float64
 2   Monetary value  10000 non-null  float64
 3   Point usage     10000 non-null  float64
dtypes: float64(4)
memory usage: 390.6 KB
None


In [14]:
# установка индексов
customers_analytics = customers_analytics.reset_index() # индекс по умолчанию
print(customers_analytics.head())

   CustomerID  Recency  Frequency  Monetary value  Point usage
0     2898197     64.0   0.964286     2695.222222       4754.0
1     2903215    367.0   0.285714     1697.875000       1057.0
2     2748887     23.0   1.833333     1241.320000       7020.0
3     2862077    761.0   0.724138      565.000000       1123.0
4     2801997    458.0   0.517241      580.033333       1347.5


In [15]:
#изъятие информации о возрасте и поле клиентов
customers_info = first_file[["CustomerId","Age","Sex"]] 
print(customers_info.head())

   CustomerId   Age     Sex
0     2728183  21.0  Female
1     2728198  21.0  Female
2     2728306  21.0  Female
3     2728178  22.0    Male
4     2728322  22.0    Male


In [16]:
# соединение двух таблиц
data_for_model = pd.merge(left = customers_analytics, right = customers_info, left_on="CustomerID", right_on="CustomerId")
# соединяем результаты recency,frequency и тд и инфу о возрасте и поле
data_for_model.drop("CustomerId", axis=1, inplace = True) # удаляем указанную метку из столбцов
mean_Age = data_for_model['Age'].mean() # среднее значение возраста 
data_for_model['Age'] = data_for_model.Age.fillna(mean_Age) # заполняем значения NaN средней датой
data_for_model.Sex.value_counts(dropna=False) # отображаем количество различных значений

Female    5141
Male      3733
NaN       1126
Name: Sex, dtype: int64

In [17]:
# декодинг пола клиента 
#def sex_to_numbers(sex):
#    if sex == 'Male':
#        return 1
#    elif sex == 'Female':
#        return 2
#    else:
#        return 0

In [18]:
# формирование обучающей и тестовой выборки
#data_for_model.Sex = data_for_model.Sex.apply(sex_to_numbers)
label_encoder = LabelEncoder() # подключаем LabelEncoder
encode_values = label_encoder.fit_transform(data_for_model['Sex'].astype(str)) # преобразуем значения в 0 и 1 для Male и Female 
onehot_encoder = OneHotEncoder(sparse=False) # возвращаем массив
encode_values = encode_values.reshape(len(encode_values),1) # меняем форму массива 
onehot_encoder = onehot_encoder.fit_transform(encode_values) # преобразуем столбец в 3 новых (для male,female,NaN)
x = data_for_model[['Recency','Frequency','Monetary value','Point usage','Age']].to_numpy() # преобразуем в numpy array
x = np.column_stack([x,onehot_encoder]) # соединяем результаты декодинга и это
print(x)
unique_people = second_file.groupby(by='CustomerID').BetweenTwoDates.max() # чтобы найти уникальные значения
y = unique_people.values
print(x.size)
print(y.size)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42) 
# х - отобранные данные, у - покупатели, 0.2 - тестовая выборка, 42 - разделяем данные на обучающие и тестовые

[[6.40000000e+01 9.64285714e-01 2.69522222e+03 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.67000000e+02 2.85714286e-01 1.69787500e+03 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.30000000e+01 1.83333333e+00 1.24132000e+03 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [8.83000000e+02 1.00000000e-01 2.45000000e+02 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.06000000e+02 4.16666667e-01 2.46600000e+02 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.01000000e+02 1.86666667e+00 6.07125000e+02 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]]
80000
10000


In [19]:
# обучение
regression = LogisticRegression() # логистическая регрессия
regression.fit(x_train, y_train) # обучение на логистической регрессии

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
# вероятность посещения на тестовой выборке
y_predicted = regression.predict(x_test) # предсказываем метки
print(confusion_matrix(y_test,y_predicted)) # подводим итоги обучения ()
print(regression.score(x_test, y_test)) # средняя точность данных

[[1065   14]
 [ 908   13]]
0.539


In [112]:
# подсчёт precision и recall
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

       False       0.54      0.99      0.70      1079
        True       0.48      0.01      0.03       921

    accuracy                           0.54      2000
   macro avg       0.51      0.50      0.36      2000
weighted avg       0.51      0.54      0.39      2000

